In [0]:
import sys
import time
import datetime
import os
import math
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.pyplot import legend

plt.rcParams['figure.figsize'] = [20, 15]


# From Handbook of Mathematical Functions, formula 7.1.26.
def erf(x):
    # save the sign of x
    sign = 1 if x >= 0 else -1
    x = abs(x)

    # constants
    a1 =  0.254829592
    a2 = -0.284496736
    a3 =  1.421413741
    a4 = -1.453152027
    a5 =  1.061405429
    p  =  0.3275911

    # A&S formula 7.1.26
    t = 1.0/(1.0 + p*x)
    y = 1.0 - (((((a5*t + a4)*t) + a3)*t + a2)*t + a1)*t*math.exp(-x*x)
    return sign*y # erf(-x) = -erf(x)

def polynomial_fitted_y_values(x, w):
    x = [x ** j for j in np.arange(0, w.size)]
    wTx = w[:, np.newaxis].T.dot(x).T
    return wTx[:, 0]

def least_squares(M, x, t):
    x = np.array([x ** j for j in np.arange(0, M + 1)])
    return np.linalg.solve(x.dot(x.T), x.dot(t))

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mean_absolute_error(predictions, targets):
  return np.absolute(predictions - targets).mean()

def absolute_error(predictions, targets):
  return np.absolute(predictions - targets).sum()
  
def main():
  
    x = np.linspace(-10, 10, 21)
    
    t =  np.array([erf(xVal) for xVal in x])
    xs = np.linspace(-10, 10, 21)
    
    t_ideal = np.array([erf(xVal) for xVal in xs])
    
    l2_diff = []
    rms_diff = []
    mae_diff = []
    abs_errors = []

    M = np.arange(0, 10).reshape(-1)
    fig_row = np.ceil(M.shape[0] / 2)
    fig_col = np.ceil(len(M) / fig_row)
    fig = plt.figure()
    for i, m in enumerate(M):
        w = least_squares(m, x, t)
        fig.add_subplot(fig_row, fig_col, i + 1)
        plt.plot(x, t, 'b.', label='Training data')
        plt.plot(xs, t_ideal, 'g-', label='f(x) = erf(x)')
        # Compute Various errors here
        l2_norm = np.linalg.norm((t_ideal - polynomial_fitted_y_values(xs, w)), 2)
        rms_error = rmse(polynomial_fitted_y_values(xs, w), t_ideal)
        mae = mean_absolute_error(polynomial_fitted_y_values(xs, w), t_ideal)
        abs_error = absolute_error(polynomial_fitted_y_values(xs, w), t_ideal)
        
        rms_diff.append(rms_error)
        mae_diff.append(mae)
        abs_errors.append(abs_error)
        
        plt.plot(xs, polynomial_fitted_y_values(xs, w), 'y-', label='Polynomial Curve fitting')
        plt.legend()
        plt.title('M = {0}'.format(m))
        plt.xlim(-10, 10)
        plt.ylim(-6.5, 6.5)

    fig, ax = plt.subplots(figsize=(10, 5))   
    ax.plot(np.arange(0, 10), abs_errors, marker='o', label='Absolute Maximum Error')
    ax.plot(np.arange(0, 10), rms_diff, marker="*", label='RMSE')
    ax.set_title('Degree vs. RMS Error and Absolute Maximum Error')
    ax.set_xlabel('n = Degree')
    ax.set_ylabel('Error')
    plt.legend()

    plt.yticks(np.arange(-10,10, 2))
    plt.xticks(np.arange(0,10,1))
    plt.show()

if __name__ == '__main__':
  main()